<a href="https://colab.research.google.com/github/seven320/-Target_on_with_Arduino/blob/master/FP058_short_former.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jan 24 12:24:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# ENV

In [2]:
import os, sys
COLAB = "google.colab" in sys.modules

if COLAB:
    from google.colab import drive
    from google.colab import output
    # drive._mount("/content/drive")

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [4]:
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

/content
/content/drive
/content
/


In [5]:
import re
if COLAB:
    COMPE_NAME = "FeedbackPrize"
    BASE_DIR = f"/content/drive/MyDrive/kaggle/{COMPE_NAME}"
    NOTEBOOK_NAME = "FP058-short-former"
    VER = re.sub(r"\D", "", NOTEBOOK_NAME)[:3]
else:
    BASE_DIR = ".."

In [6]:
if COLAB:
    !pip install transformers > /dev/null
    !pip install einops > /dev/null
    !pip install timm > /dev/null
    !pip install kaggle > /dev/null
    !pip install kaggle_datasets > /dev/null
    !pip install git+https://github.com/albumentations-team/albumentations
    !pip install tensorflow-determinism
    output.clear()

In [7]:
import gc
import json
import math
import os
import random
import re
import time
import copy
import warnings
from contextlib import contextmanager

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from torch.cuda.amp import autocast
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold, train_test_split, KFold
from sklearn.preprocessing import RobustScaler
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from einops.layers.torch import Rearrange, Reduce
import timm
from sklearn.metrics import accuracy_score

from tqdm.notebook import tqdm
from transformers import AdamW, get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig, TFAutoModel
from typing import Optional, Tuple
from xgboost import XGBRegressor
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print('TF version',tf.__version__)

TF version 2.7.0


In [8]:
warnings.simplefilter('ignore')

In [9]:
if COLAB:
    INPUT_DIR = Path(os.path.join(BASE_DIR ,f"input"))
    INPUT_DIR.mkdir(exist_ok=True, parents=True)

    SAVE_DIR = Path(os.path.join(BASE_DIR ,f"models/{NOTEBOOK_NAME}"))
    SAVE_DIR.mkdir(exist_ok=True, parents=True)

    OOF_DIR = Path(os.path.join(BASE_DIR, f"oof/{NOTEBOOK_NAME}"))
    OOF_DIR.mkdir(exist_ok=True, parents = True)

    SUB_DIR = Path(os.path.join(BASE_DIR, f"submission/{NOTEBOOK_NAME}"))
    SUB_DIR.mkdir(exist_ok=True, parents = True)

else:
    INPUT_DIR = "../input/feedback-prize-2021/train.csv"

In [23]:
# KAGGLE ONLY HAS 1, BUT OFFLINE, YOU CAN USE MORE
os.environ["CUDA_VISIBLE_DEVICES"]="0" #0,1,2,3 for four gpu

# tokenが存在するときは使用
LOAD_TOKENS_FROM = None
# os.path.join(BASE_DIR, "models/FP030-TF")

DOWNLOADED_MODEL_PATH = None

LOAD_MODEL_FROM = None

if DOWNLOADED_MODEL_PATH is None:
    # DOWNLOADED_MODEL_PATH = 'allenai/longformer-base-4096'    
    DOWNLOADED_MODEL_PATH = "roberta-base"
# MODEL_NAME = 'allenai/longformer-base-4096'
MODEL_NAME = "roberta-base"

In [24]:
# USE MULTIPLE GPUS
if os.environ["CUDA_VISIBLE_DEVICES"].count(',') == 0:
    strategy = tf.distribute.get_strategy()
    print('single strategy')
else:
    strategy = tf.distribute.MirroredStrategy()
    print('multiple strategy')

single strategy


In [25]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
print('Mixed precision enabled')

Mixed precision enabled


In [26]:
train_df = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"))
sample_df = pd.read_csv(os.path.join(INPUT_DIR, "sample_submission.csv"))

In [27]:
#############
## Utility ##
#############
@contextmanager
def timer(name: str):
    t0 = time.time()
    print(f"[{name}] start")
    yield
    print(f"[{name}] done - elapsed {time.time() - t0:.2f}s")

def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results
    
    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = str(seed) # set tf seed
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    tf.random.set_seed(seed)

# seed_everything(42)

# config

In [28]:
from torch import cuda
config = {'model_name': MODEL_NAME,   
         'max_length': 1024,
         'train_batch_size':4,
         'valid_batch_size':16,
         'epochs':5,
         'learning_rate': 5e-5,
         'max_grad_norm':10,
         'device': 'cuda' if cuda.is_available() else 'cpu',
         'train_fold': [0],
         'n_fold': 5,
         'debug': False,
          }

# THIS WILL COMPUTE VAL SCORE DURING COMMIT BUT NOT DURING SUBMIT
COMPUTE_VAL_SCORE = True

if COLAB:
    TRAIN_FILES = Path(os.path.join(BASE_DIR, f"input/train"))
else:
    TRAIN_FILES = Path('../input/feedback-prize-2021/train')
if COLAB:
    TEST_FILES = Path(os.path.join(BASE_DIR, f"input/test"))
else:
    TEST_FILES = Path('../input/feedback-prize-2021/test')

if len(os.listdir(TEST_FILES)) > 5:
    COMPUTE_VAL_SCORE = False

# Train準備

In [29]:
if DOWNLOADED_MODEL_PATH == 'model':
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.save_pretrained(SAVE_DIR)

    config_model = AutoConfig.from_pretrained(MODEL_NAME) 
    config_model.save_pretrained(SAVE_DIR)

    backbone = TFAutoModel.from_pretrained(MODEL_NAME, config=config_model)
    backbone.save_pretrained(SAVE_DIR)

In [30]:
IDS = train_df.id.unique()
print('There are',len(IDS),'train texts.')

There are 15594 train texts.


In [31]:
# MAX_LEN = 1024
MAX_LEN = 512

# THE TOKENS AND ATTENTION ARRAYS
tokenizer = AutoTokenizer.from_pretrained(DOWNLOADED_MODEL_PATH)
train_tokens = np.zeros((len(IDS),MAX_LEN), dtype='int32')
train_attention = np.zeros((len(IDS),MAX_LEN), dtype='int32')

# THE 14 CLASSES FOR NER
lead_b = np.zeros((len(IDS),MAX_LEN))
lead_i = np.zeros((len(IDS),MAX_LEN))

position_b = np.zeros((len(IDS),MAX_LEN))
position_i = np.zeros((len(IDS),MAX_LEN))

evidence_b = np.zeros((len(IDS),MAX_LEN))
evidence_i = np.zeros((len(IDS),MAX_LEN))

claim_b = np.zeros((len(IDS),MAX_LEN))
claim_i = np.zeros((len(IDS),MAX_LEN))

conclusion_b = np.zeros((len(IDS),MAX_LEN))
conclusion_i = np.zeros((len(IDS),MAX_LEN))

counterclaim_b = np.zeros((len(IDS),MAX_LEN))
counterclaim_i = np.zeros((len(IDS),MAX_LEN))

rebuttal_b = np.zeros((len(IDS),MAX_LEN))
rebuttal_i = np.zeros((len(IDS),MAX_LEN))

# HELPER VARIABLES
train_lens = []
targets_b = [lead_b, position_b, evidence_b, claim_b, conclusion_b, counterclaim_b, rebuttal_b]
targets_i = [lead_i, position_i, evidence_i, claim_i, conclusion_i, counterclaim_i, rebuttal_i]
target_map = {'Lead':0, 'Position':1, 'Evidence':2, 'Claim':3, 'Concluding Statement':4,
             'Counterclaim':5, 'Rebuttal':6}

In [32]:
# WE ASSUME DATAFRAME IS ASCENDING WHICH IT IS
assert(np.sum(train_df.groupby('id')['discourse_start'].diff()<=0)==0)

# FOR LOOP THROUGH EACH TRAIN TEXT
for id_num in tqdm(range(len(IDS))):
    if LOAD_TOKENS_FROM: break
    # READ TRAIN TEXT, TOKENIZE, AND SAVE IN TOKEN ARRAYS    
    n = IDS[id_num]
    name = os.path.join(BASE_DIR, f'input/train/{n}.txt')
    txt = open(name, 'r').read()
    train_lens.append(len(txt.split()))
    tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN, padding='max_length',
                                   truncation=True, return_offsets_mapping=True)
    train_tokens[id_num,] = tokens['input_ids']
    train_attention[id_num,] = tokens['attention_mask']
    
    # FIND TARGETS IN TEXT AND SAVE IN TARGET ARRAYS
    offsets = tokens['offset_mapping']
    offset_index = 0
    df = train_df.loc[train_df.id==n]
    for index,row in df.iterrows():
        a = row.discourse_start
        b = row.discourse_end
        if offset_index>len(offsets)-1:
            break
        c = offsets[offset_index][0]
        d = offsets[offset_index][1]
        beginning = True
        while b>c:
            if (c>=a)&(b>=d):
                k = target_map[row.discourse_type]
                if beginning:
                    targets_b[k][id_num][offset_index] = 1
                    beginning = False
                else:
                    targets_i[k][id_num][offset_index] = 1
            offset_index += 1
            if offset_index>len(offsets)-1:
                break
            c = offsets[offset_index][0]
            d = offsets[offset_index][1]

  0%|          | 0/15594 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
if LOAD_TOKENS_FROM is None:
    plt.hist(train_lens,bins=100)
    plt.title('Histogram of Train Word Counts',size=16)
    plt.xlabel('Train Word Count',size=14)
    plt.show()

In [ ]:
if LOAD_TOKENS_FROM is None:
    targets = np.zeros((len(IDS),MAX_LEN,15), dtype='int32')
    for k in range(7):
        targets[:,:,2*k] = targets_b[k]
        targets[:,:,2*k+1] = targets_i[k]
    targets[:,:,14] = 1-np.max(targets,axis=-1)

In [ ]:
if LOAD_TOKENS_FROM is None:
    np.save(os.path.join(SAVE_DIR, f'targets_{MAX_LEN}'), targets)
    np.save(os.path.join(SAVE_DIR, f'tokens_{MAX_LEN}'), train_tokens)
    np.save(os.path.join(SAVE_DIR, f'attention_{MAX_LEN}'), train_attention)
    print('Saved NER tokens')
else:
    targets = np.load(f'{LOAD_TOKENS_FROM}/targets_{MAX_LEN}.npy')
    train_tokens = np.load(f'{LOAD_TOKENS_FROM}/tokens_{MAX_LEN}.npy')
    train_attention = np.load(f'{LOAD_TOKENS_FROM}/attention_{MAX_LEN}.npy')
    print('Loaded NER tokens')

In [ ]:
targets.shape

In [ ]:
def build_model():
    tokens = tf.keras.layers.Input(shape=(MAX_LEN,), name = 'tokens', dtype=tf.int32)
    attention = tf.keras.layers.Input(shape=(MAX_LEN,), name = 'attention', dtype=tf.int32)
    
    model_config = AutoConfig.from_pretrained(DOWNLOADED_MODEL_PATH) 
    backbone = TFAutoModel.from_pretrained(DOWNLOADED_MODEL_PATH, config=model_config)
    
    x = backbone(tokens, attention_mask=attention)
    x = tf.keras.layers.Dense(256, activation='relu')(x[0])
    x = tf.keras.layers.Dense(15, activation='softmax', dtype='float32')(x)

    model = tf.keras.Model(inputs=[tokens,attention], outputs=x)
    model.compile(optimizer = tf.keras.optimizers.Adam(lr=1e-4),
                  loss = [tf.keras.losses.CategoricalCrossentropy()],
                  metrics = [tf.keras.metrics.CategoricalAccuracy()],
                  sample_weight_mode='temporal')
    
    return model

In [ ]:
with strategy.scope():
    model = build_model()

In [ ]:
target_map_rev = {0:'Lead', 1:'Position', 2:'Evidence', 3:'Claim', 4:'Concluding Statement',
             5:'Counterclaim', 6:'Rebuttal', 7:'blank'}

In [ ]:
def get_preds(dataset='train', verbose=True, text_ids=None, preds=None):
    all_predictions = []

    for id_num in range(len(preds)):
    
        # GET ID
        if (id_num%1000==0)&(verbose): 
            print(id_num,', ',end='')
        n = text_ids[id_num]
    
        # GET TOKEN POSITIONS IN CHARS
        name = os.path.join(BASE_DIR, f'input/train/{n}.txt')
        txt = open(name, 'r').read()
        tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN, padding='max_length',
                                   truncation=True, return_offsets_mapping=True)
        off = tokens['offset_mapping']
    
        # GET WORD POSITIONS IN CHARS
        w = []
        blank = True
        for i in range(len(txt)):
            if (txt[i]!=' ')&(txt[i]!='\n')&(txt[i]!='\xa0')&(txt[i]!='\x85')&(blank==True):
                w.append(i)
                blank=False
            elif (txt[i]==' ')|(txt[i]=='\n')|(txt[i]=='\xa0')|(txt[i]=='\x85'):
                blank=True
        w.append(1e6)
            
        # MAPPING FROM TOKENS TO WORDS
        word_map = -1 * np.ones(MAX_LEN,dtype='int32')
        w_i = 0
        for i in range(len(off)):
            if off[i][1]==0: continue
            while off[i][0]>=w[w_i+1]: w_i += 1
            word_map[i] = int(w_i)
        
        # CONVERT TOKEN PREDICTIONS INTO WORD LABELS
        ### KEY: ###
        # 0: LEAD_B, 1: LEAD_I
        # 2: POSITION_B, 3: POSITION_I
        # 4: EVIDENCE_B, 5: EVIDENCE_I
        # 6: CLAIM_B, 7: CLAIM_I
        # 8: CONCLUSION_B, 9: CONCLUSION_I
        # 10: COUNTERCLAIM_B, 11: COUNTERCLAIM_I
        # 12: REBUTTAL_B, 13: REBUTTAL_I
        # 14: NOTHING i.e. O
        ### NOTE THESE VALUES ARE DIVIDED BY 2 IN NEXT CODE LINE
        pred = preds[id_num,]/2.0
    
        i = 0
        while i<MAX_LEN:
            prediction = []
            start = pred[i]
            if start in [0,1,2,3,4,5,6,7]:
                prediction.append(word_map[i])
                i += 1
                if i>=MAX_LEN: break
                while pred[i]==start+0.5:
                    if not word_map[i] in prediction:
                        prediction.append(word_map[i])
                    i += 1
                    if i>=MAX_LEN: break
            else:
                i += 1
            prediction = [x for x in prediction if x!=-1]
            if len(prediction)>4:
                all_predictions.append( (n, target_map_rev[int(start)], 
                                ' '.join([str(x) for x in prediction]) ) )
                
    # MAKE DATAFRAME
    if len(all_predictions) == 0: # fake
       all_predictions = [[0, "lead", "1"], [1, "lead", "3"]]
    df = pd.DataFrame(all_predictions)
    df.columns = ['id','class','predictionstring']
    
    return df

In [ ]:
# CODE FROM : Rob Mulla @robikscube
# https://www.kaggle.com/robikscube/student-writing-competition-twitch
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]

def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])

    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

In [ ]:
def calc_train_score(model):
    train_idx_cp = train_idx[:len(valid_idx)]

    p = model.predict([train_tokens[train_idx_cp,], train_attention[train_idx_cp,]], 
                  batch_size=config["valid_batch_size"], verbose=2)
    print('TRAIN predictions shape:',p.shape)
    oof_preds = np.argmax(p,axis=-1)

    oof = get_preds( dataset='train', verbose=True, text_ids=IDS[train_idx_cp], preds=oof_preds)
    oof.head()
    # TRAIN DATAFRAME
    train = train_df.loc[train_df['id'].isin(IDS[train_idx_cp])]

    f1s = []
    CLASSES = oof['class'].unique()
    score_detail = {}
    print()
    for c in CLASSES:
        pred_df = oof.loc[oof['class']==c].copy()
        gt_df = train.loc[train['discourse_type']==c].copy()
        f1 = score_feedback_comp(pred_df, gt_df)
        print(f"{c}:{f1:.4f}")
        f1s.append(f1)
        score_detail[c] = f1
    print(f'Overall: {np.mean(f1s):.4f}')
    print()
    return score_detail, np.mean(f1s)

def calc_oof_score(model):
    p = model.predict([train_tokens[valid_idx,], train_attention[valid_idx,]], 
                  batch_size=config["valid_batch_size"], verbose=2)
    print('OOF predictions shape:',p.shape)
    oof_preds = np.argmax(p,axis=-1)

    oof = get_preds( dataset='train', verbose=True, text_ids=IDS[valid_idx], preds=oof_preds)
    oof.head()
    # VALID DATAFRAME
    valid = train_df.loc[train_df['id'].isin(IDS[valid_idx])]

    f1s = []
    score_detail = {}
    CLASSES = oof['class'].unique()
    print()
    for c in CLASSES:
        pred_df = oof.loc[oof['class']==c].copy()
        gt_df = valid.loc[valid['discourse_type']==c].copy()
        f1 = score_feedback_comp(pred_df, gt_df)
        print(f"{c}:{f1:.4f}")
        f1s.append(f1)
        score_detail[c] = f1
    print(f'Overall: {np.mean(f1s):.4f}')
    print()
    return score_detail, np.mean(f1s)

# log

In [ ]:
class Log():
    def __init__(self):
        self.train_losses = []
        self.valid_losses = []
        self.train_scores = []
        self.valid_scores = []
        self.train_score_details = []
        self.valid_score_details = []

    def add_scores(self, train_score, valid_score):
        self.train_scores.append(train_score)
        self.valid_scores.append(valid_score)

    def add_class_scores(self, train_score_detail, valid_score_detail):
        self.train_score_details.append(train_score_detail)
        self.valid_score_details.append(valid_score_detail)

In [ ]:
def show_log(log: Log, fold):
    plt.figure(figsize=(15, 5))
    # plt2 = plt.add_subplot(111)
    plt.plot(
        np.arange(len(log.train_scores)),
        log.train_scores,
        "-o",
        label="Train score",
        color="#2ca02c")
    plt.plot(
        np.arange(len(log.train_scores)),
        log.valid_scores,
        "-o",
        label="Val score",
        color="#d62728")

    ydist = plt.ylim()[1] - plt.ylim()[0]

    plt.ylabel("Loss", size=14)
    plt.title(f"Fold {fold}", size=18)

    plt.legend(loc=3)
    plt.savefig(OOF_DIR / f"fig{fold}.png")
    plt.show()

def show_detail_log(log: Log, fold):
    plt.figure(figsize=(15, 5))
    plt2 = plt.gca().twinx()

    classes = []
    for score_detail in log.train_score_details:
        classes += list(score_detail.keys())
    
    classes = list(set(classes))
    train_class_scores, valid_class_scores = {}, {}
    for epoch in range(len(log.train_score_details)):
        for c in classes:
            if epoch == 0:
                train_class_scores[c] = []
                valid_class_scores[c] = []
            train_class_scores[c].append(log.train_score_details[epoch].get(c, 0))
            valid_class_scores[c].append(log.valid_score_details[epoch].get(c, 0))

    colorlist = ["r", "g", "b", "c", "m", "y", "k"]
    for i, c in enumerate(classes):
        plt2.plot(
            np.arange(len(log.train_scores)),
            train_class_scores[c],
            "-o",
            label=f"{c}_train",
            color = colorlist[i]
            )
        plt2.plot(
            np.arange(len(log.train_scores)),
            valid_class_scores[c],
            "-w",
            linestyle="dashdot",
            label=f"{c}_valid",
            color = colorlist[i]
            )

    ydist = plt.ylim()[1] - plt.ylim()[0]

    plt.ylabel("Loss", size=14)
    plt.title(f"Fold {fold + 1}", size=18)

    plt.legend(loc=3)
    plt.savefig(OOF_DIR / f"fig{fold}_detail.png")
    plt.show()

In [ ]:
seed_everything(42)
kf = KFold(n_splits = config["n_fold"], shuffle = True, random_state = 200033)
train_df["fold"] = 0
if config["debug"]:
    train_df = train_df[:1000]
    IDS = IDS[:1000]

for fold, (train_idx, valid_idx) in enumerate(kf.split(IDS)):
    train_df.loc[train_df["id"].isin(IDS[valid_idx]), "fold"] = fold

    p = targets[valid_idx]
    target_cnts = p.sum(axis=(0,1))
    print(target_cnts)

In [ ]:
# 0, 1の特殊なtoken以外の部分でランダムにtokenを書き換える
def eraser(input_tokens, p = 0.5, del_ratio = 0.03):
    # 0.5の確率で文章中の3文字の連続した単語をランダムな単語に書き換える
    input_tokens = copy.deepcopy(input_tokens)
    assert input_tokens.ndim == 2
    if input_tokens.ndim == 2:
        batch_size, length = input_tokens.shape

    for b in range(batch_size):
        p_1 = np.random.rand()
        if p_1 > p:
            continue
        # 連続del_words削除
        del_words = int(np.sum(input_tokens[b] > 1) * del_ratio)
        # print(del_words)
        while True:
            position = np.random.randint(1, length - del_words)
            if np.all(input_tokens[b, position:position+del_words] > 1):  
                input_tokens[b, position:position+del_words] = np.random.randint(2, 50253, del_words)
                break
    return input_tokens

# Train

In [ ]:
for fold in range(config["n_fold"]):
    print(f'Fold-{fold}/{config["n_fold"]}')
    if fold not in config["train_fold"]:
        continue

    with strategy.scope():
        model = build_model()

    # LOAD MODEL
    if LOAD_MODEL_FROM:
        model.load_weights(f'{LOAD_MODEL_FROM}/FP{VER}_model_fold{fold}.h5')
    # OR TRAIN MODEL
    else:
        log = Log()
        LRS = [0.25e-4, 0.25e-4, 0.25e-4, 0.25e-4, 0.25e-5]
        best_valid_score = 0
        for epoch in range(config["epochs"]):
            print(f"epoch {epoch + 1}/{config['epochs']}")
            def lrfn(epoch):
                return LRS[epoch]
            
            lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

            erased_train_tokens = eraser(train_tokens, del_ratio = 0.2)
            model.fit(x = [erased_train_tokens[train_idx,], train_attention[train_idx,]],
                y = targets[train_idx,],
                validation_data = ([train_tokens[valid_idx,], train_attention[valid_idx,]], targets[valid_idx,]),
                callbacks = lr_callback,
                epochs = 1,
                batch_size = config["train_batch_size"],
                verbose = 1
                )    
            LRS.pop(0)

            # TRAIN score
            train_score_detail, train_score = calc_train_score(model)
            # OOF
            valid_score_detail, valid_score = calc_oof_score(model)

            log.add_class_scores(train_score_detail, valid_score_detail)
            log.add_scores(train_score, valid_score)

            # SAVE MODEL WEIGHTS
            if valid_score > best_valid_score:
                print(f"update best score!! {best_valid_score:.4f}→{valid_score:.4f}")
                best_valid_score = valid_score
                model.save_weights(os.path.join(SAVE_DIR, f'FP{VER}_model_fold{fold}.h5'))
                print(f"model save valid score:{valid_score:.4f}")

    show_log(log, fold)
    show_detail_log(log, fold)

    print(f"best valid socre: {best_valid_score:.4f}")